In [7]:
import os

import numpy as np
np.set_printoptions(suppress=True)

from tqdm import tqdm

from keras.models import Model
from keras.preprocessing.image import load_img,img_to_array
import tensorflow.keras.backend as K

from sklearn.model_selection import StratifiedKFold
from sklearn.svm import SVC
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import GridSearchCV

import pickle


!pip install git+https://github.com/rcmalli/keras-vggface.git

import keras_vggface
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
from keras_vggface.utils import decode_predictions

  Cloning https://github.com/rcmalli/keras-vggface.git to c:\users\giulia~1\appdata\local\temp\pip-req-build-id0lnflw
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp37-none-any.whl size=8385 sha256=5e07c27e2bd803b89d663556079d892b0434b455be03dd7332e91ba0841c2ea9
  Stored in directory: C:\Users\GIULIA~1\AppData\Local\Temp\pip-ephem-wheel-cache-xj5k5t4r\wheels\36\07\46\06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface


  Running command git clone -q https://github.com/rcmalli/keras-vggface.git 'C:\Users\GIULIA~1\AppData\Local\Temp\pip-req-build-id0lnflw'


In [4]:
X_folder = 'C:/Users/GiuliaChiaretti/Desktop/Face_Detected_Dataset_3class'
os.listdir(X_folder) 

['Altro', 'Federica', 'Giulia']

### Creation X, y from folder

In [5]:
X = []
y= []
person_rep = dict()
person_folders = os.listdir(X_folder)

for i, person in enumerate(person_folders):
    person_rep[i] = person
    image_names = os.listdir(X_folder + '/' + person + '/')
    
    for image_name in tqdm(image_names): 
        img = load_img(X_folder + '/' + person + '/'+ image_name, target_size=(224,224))
        img = img_to_array(img)
        img = np.expand_dims(img, axis = 0)

        X.append(np.squeeze(K.eval(img)).tolist())
        y.append(str(person))

100%|██████████████████████████████████████████████████████████████████████████████| 1181/1181 [32:36<00:00,  1.66s/it]


In [6]:
X=np.array(X).astype('float64')
y=np.array(y)

In [14]:
np.unique(y)

array(['Altro', 'Federica', 'Giulia'], dtype='<U8')

### Model feature extraction - vggface

In [8]:
base_model = VGGFace(include_top = False, input_shape = (224, 224, 3), model='senet50')

In [9]:
model = Model(inputs=base_model.input, outputs=base_model.get_layer("global_average_pooling2d_12").output)

In [10]:
X = preprocess_input(X,version=2)

In [11]:
X_feature = model.predict(X)

In [56]:
X_feature.shape

(3618, 1024)

### SVM with 5 fold cross validation and HPO

In [57]:
param_grid = {'C': [1e2, 5e2, 1e3], 
          'gamma': [0.005, 0.001, 0.01, 0.1, 0.5, 1.0], }

clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced',probability=True), param_grid, scoring = 'accuracy', cv=5)

clf = clf.fit(X_feature,y)

print("Best parameter:")
print(" C: "+str(clf.best_estimator_.C))
print(" gamma: "+str(clf.best_estimator_.gamma))

Best parameter:
 C: 100.0
 gamma: 0.005


In [59]:
clf.best_score_

0.9997236042012161

In [60]:
#save model 

pickle.dump(clf,open( 'gridclf.sav', 'wb'))
